In [25]:
from torchvision import datasets, transforms
import torch.nn.functional as F


DATASET_DIR = "../Rock-Paper-Scissors/train"

transform = transforms.Compose([
    transforms.Resize((100, 100)),   
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(DATASET_DIR, transform=transform)

print("Classi:", dataset.classes)
print("Numero immagini:", len(dataset))


Classi: ['paper', 'rock', 'scissors']
Numero immagini: 2520


CONTROLLO LE DIMENSIONI DELLE IMMAGINI 

In [26]:
img, label = dataset[0]
print("Dimensione immagine:", img.shape)


Dimensione immagine: torch.Size([3, 100, 100])


CREO IL DATALOADER 

In [27]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class RPSNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        
        # calcolo automatico dimensione
        with torch.no_grad():
            x = torch.zeros(1, 3, 100, 100)  # batch fittizio
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            n_features = x.numel()
        
        self.fc1 = nn.Linear(n_features, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = RPSNet().to(DEVICE)


In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.3f}")


Epoch 1/10 - Loss: 48.485
Epoch 2/10 - Loss: 2.928
Epoch 3/10 - Loss: 0.324
Epoch 4/10 - Loss: 0.095
Epoch 5/10 - Loss: 0.045
Epoch 6/10 - Loss: 0.024
Epoch 7/10 - Loss: 0.015
Epoch 8/10 - Loss: 0.009
Epoch 9/10 - Loss: 0.006
Epoch 10/10 - Loss: 0.005


In [30]:
torch.save(model.state_dict(), "rps_model.pth")
print("Modello salvato!")


Modello salvato!
